In [0]:
import pandas as pd 
import sklearn
import keras
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
import tensorflow as tf

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.layers import LSTM

In [0]:
df=pd.read_csv('US_Accidents_Dec19.csv',header=0)

In [61]:
df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,I-75 S,R,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,Miamisburg Centerville Rd,R,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day


In [62]:
df.columns

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [63]:
df.isnull().sum()

ID                             0
Source                         0
TMC                       728071
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                  2246264
End_Lng                  2246264
Distance(mi)                   0
Description                    1
Number                   1917605
Street                         0
Side                           0
City                          83
County                         0
State                          0
Zipcode                      880
Country                        0
Timezone                    3163
Airport_Code                5691
Weather_Timestamp          36705
Temperature(F)             56063
Wind_Chill(F)            1852623
Humidity(%)                59173
Pressure(in)               48142
Visibility(mi)             65691
Wind_Direction             45101
Wind_Speed(mph)           440840
Precipitat

In [64]:
# checking the percentage of all the values
# null values = 100 - the values shown

df.count()/2974242*(100)

ID                       100.003127
Source                   100.003127
TMC                       75.523915
Severity                 100.003127
Start_Time               100.003127
End_Time                 100.003127
Start_Lat                100.003127
Start_Lng                100.003127
End_Lat                   24.479212
End_Lng                   24.479212
Distance(mi)             100.003127
Description              100.003093
Number                    35.529389
Street                   100.003127
Side                     100.003127
City                     100.000336
County                   100.003127
State                    100.003127
Zipcode                   99.973539
Country                  100.003127
Timezone                  99.896780
Airport_Code              99.811784
Weather_Timestamp         98.769031
Temperature(F)            98.118176
Wind_Chill(F)             37.714214
Humidity(%)               98.013612
Pressure(in)              98.384496
Visibility(mi)            97

In [65]:
# Convert Start_Time and End_Time to datetypes

df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')
df['End_Time'] = pd.to_datetime(df['End_Time'], errors='coerce')

# Extract year, month, day, hour and weekday

df['Year']=df['Start_Time'].dt.year
df['Month']=df['Start_Time'].dt.strftime('%b')
df['Day']=df['Start_Time'].dt.day
df['Hour']=df['Start_Time'].dt.hour
df['Weekday']=df['Start_Time'].dt.strftime('%a')

# Extract the amount of time in the unit of minutes for each accident, round to the nearest integer

td='Time_Duration(min)'
df[td]=round((df['End_Time']-df['Start_Time'])/np.timedelta64(1,'m'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2974335 entries, 0 to 2974334
Data columns (total 55 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     object        
 1   Source                 object        
 2   TMC                    float64       
 3   Severity               int64         
 4   Start_Time             datetime64[ns]
 5   End_Time               datetime64[ns]
 6   Start_Lat              float64       
 7   Start_Lng              float64       
 8   End_Lat                float64       
 9   End_Lng                float64       
 10  Distance(mi)           float64       
 11  Description            object        
 12  Number                 float64       
 13  Street                 object        
 14  Side                   object        
 15  City                   object        
 16  County                 object        
 17  State                  object        
 18  Zipcode               

In [66]:
# Checking if there is any negative time_duration values

df[td][df[td]<=0]

69719      -0.0
69720      -0.0
69721      -0.0
69722      -1.0
69723      -1.0
309387    -30.0
309388    -30.0
628436    -12.0
628472    -31.0
628489     -0.0
628923    -14.0
628942     -0.0
628943    -30.0
1265261   -30.0
1265262   -31.0
2002026   -30.0
2002111   -30.0
2002112   -31.0
2565229   -34.0
2565705    -7.0
2565708    -7.0
2693474   -31.0
Name: Time_Duration(min), dtype: float64

In [67]:
# Drop the rows with td<0

neg_outliers=df[td]<=0

# Set outliers to NAN

df[neg_outliers] = np.nan

# Drop rows with negative td

df.dropna(subset=[td],axis=0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2974313 entries, 0 to 2974334
Data columns (total 55 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     object        
 1   Source                 object        
 2   TMC                    float64       
 3   Severity               float64       
 4   Start_Time             datetime64[ns]
 5   End_Time               datetime64[ns]
 6   Start_Lat              float64       
 7   Start_Lng              float64       
 8   End_Lat                float64       
 9   End_Lng                float64       
 10  Distance(mi)           float64       
 11  Description            object        
 12  Number                 float64       
 13  Street                 object        
 14  Side                   object        
 15  City                   object        
 16  County                 object        
 17  State                  object        
 18  Zipcode               

In [68]:
# Double check to make sure no more negative td

df[td][df[td]<=0]

Series([], Name: Time_Duration(min), dtype: float64)

In [69]:
# Remove outliers for Time_Duration(min): n * standard_deviation (n=3), backfill with median

n=3

median = df[td].median()
std = df[td].std()
outliers = (df[td] - median).abs() > std*n

# Set outliers to NAN

df[outliers] = np.nan

# Fill NAN with median

df[td].fillna(median, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2974313 entries, 0 to 2974334
Data columns (total 55 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     object        
 1   Source                 object        
 2   TMC                    float64       
 3   Severity               float64       
 4   Start_Time             datetime64[ns]
 5   End_Time               datetime64[ns]
 6   Start_Lat              float64       
 7   Start_Lng              float64       
 8   End_Lat                float64       
 9   End_Lng                float64       
 10  Distance(mi)           float64       
 11  Description            object        
 12  Number                 float64       
 13  Street                 object        
 14  Side                   object        
 15  City                   object        
 16  County                 object        
 17  State                  object        
 18  Zipcode               

In [70]:
# Print time_duration information

print('Max time to clear an accident: {} minutes or {} hours or {} days; Min to clear an accident td: {} minutes.'.format(df[td].max(),round(df[td].max()/60), round(df[td].max()/60/24), df[td].min()))

Max time to clear an accident: 11080.0 minutes or 185 hours or 8 days; Min to clear an accident td: 1.0 minutes.


In [71]:
df.columns

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight', 'Year', 'Month', 'Day', 'Hour', 'Weekday',
       'Time_Duration(min)'],
      dtype='object')

In [0]:
# Set the list of features to include in Machine Learning

df1=df[['Source','TMC','Severity','Start_Lng','Description','Wind_Speed(mph)','Astronomical_Twilight',
             'Start_Lat','Start_Lng','Distance(mi)','Side','City','County','Civil_Twilight',
             'State','Timezone','Temperature(F)','Humidity(%)','Nautical_Twilight',
             'Pressure(in)', 'Visibility(mi)', 'Wind_Direction','Weather_Condition',
             'Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout',
             'Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop','Sunrise_Sunset',
             'Hour','Weekday', 'Time_Duration(min)','Year', 'Month', 'Day',]]

In [73]:
# Check missing values

df1.isnull().mean()

Source                   0.000196
TMC                      0.244799
Severity                 0.000196
Start_Lng                0.000196
Description              0.000196
Wind_Speed(mph)          0.148376
Astronomical_Twilight    0.000227
Start_Lat                0.000196
Start_Lng                0.000196
Distance(mi)             0.000196
Side                     0.000196
City                     0.000224
County                   0.000196
Civil_Twilight           0.000227
State                    0.000196
Timezone                 0.001259
Temperature(F)           0.019033
Humidity(%)              0.020079
Nautical_Twilight        0.000227
Pressure(in)             0.016372
Visibility(mi)           0.022271
Wind_Direction           0.015352
Weather_Condition        0.022353
Amenity                  0.000196
Bump                     0.000196
Crossing                 0.000196
Give_Way                 0.000196
Junction                 0.000196
No_Exit                  0.000196
Railway       

In [74]:
# droping the nan and null values

df1.dropna(subset=df1.columns[df1.isnull().mean()!=0], how='any', axis=0, inplace=True)
df1.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(1886924, 43)

In [75]:
# Number of null values

df1.isnull().sum().sum()

0

In [76]:
df1.head()

,Source,TMC,Severity,Start_Lng,Description,Wind_Speed(mph),Astronomical_Twilight,Start_Lat,Start_Lng,Distance(mi),Side,City,County,Civil_Twilight,State,Timezone,Temperature(F),Humidity(%),Nautical_Twilight,Pressure(in),Visibility(mi),Wind_Direction,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Hour,Weekday,Time_Duration(min),Year,Month,Day
2,MapQuest,201.0,2.0,-84.032608,Accident on OH-32 State Route 32 Westbound at ...,3.5,Day,39.063148,-84.032608,0.01,R,Williamsburg,Clermont,Night,OH,US/Eastern,36.0,100.0,Day,29.67,10.0,SW,Overcast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Night,6.0,Mon,30.0,2016.0,Feb,8.0
3,MapQuest,201.0,3.0,-84.205582,Accident on I-75 Southbound at Exits 52 52B US...,4.6,Day,39.747753,-84.205582,0.01,R,Dayton,Montgomery,Day,OH,US/Eastern,35.1,96.0,Day,29.64,9.0,SW,Mostly Cloudy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Night,7.0,Mon,30.0,2016.0,Feb,8.0
4,MapQuest,201.0,2.0,-84.188354,Accident on McEwen Rd at OH-725 Miamisburg Cen...,3.5,Day,39.627781,-84.188354,0.01,R,Dayton,Montgomery,Day,OH,US/Eastern,36.0,89.0,Day,29.65,6.0,SW,Mostly Cloudy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Day,7.0,Mon,30.0,2016.0,Feb,8.0
5,MapQuest,201.0,3.0,-82.925194,Accident on I-270 Outerbelt Northbound near Ex...,3.5,Day,40.100590,-82.925194,0.01,R,Westerville,Franklin,Day,OH,US/Eastern,37.9,97.0,Day,29.63,7.0,SSW,Light Rain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Day,7.0,Mon,30.0,2016.0,Feb,8.0
6,MapQuest,201.0,2.0,-84.230507,Accident on Oakridge Dr at Woodward Ave. Expec...,3.5,Day,39.758274,-84.230507,0.00,R,Dayton,Montgomery,Day,OH,US/Eastern,34.0,100.0,Day,29.66,7.0,WSW,Overcast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Day,7.0,Mon,30.0,2016.0,Feb,8.0


In [77]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1886924 entries, 2 to 2246263
Data columns (total 43 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Source                 object 
 1   TMC                    float64
 2   Severity               float64
 3   Start_Lng              float64
 4   Description            object 
 5   Wind_Speed(mph)        float64
 6   Astronomical_Twilight  object 
 7   Start_Lat              float64
 8   Start_Lng              float64
 9   Distance(mi)           float64
 10  Side                   object 
 11  City                   object 
 12  County                 object 
 13  Civil_Twilight         object 
 14  State                  object 
 15  Timezone               object 
 16  Temperature(F)         float64
 17  Humidity(%)            float64
 18  Nautical_Twilight      object 
 19  Pressure(in)           float64
 20  Visibility(mi)         float64
 21  Wind_Direction         object 
 22  Weather_Condition 

In [78]:
# Implementing the LabelEncoder for the Categorical values

labelencoder = LabelEncoder()
df1['Source'] = labelencoder.fit_transform(df1['Source'])
df1['Description'] = labelencoder.fit_transform(df1['Description'])
df1['Astronomical_Twilight'] = labelencoder.fit_transform(df1['Astronomical_Twilight'])
df1['Side'] = labelencoder.fit_transform(df1['Side'])
df1['City'] = labelencoder.fit_transform(df1['City'])
df1['County'] = labelencoder.fit_transform(df1['County'])
df1['Civil_Twilight'] = labelencoder.fit_transform(df1['Civil_Twilight'])
df1['State'] = labelencoder.fit_transform(df1['State'])
df1['Timezone'] = labelencoder.fit_transform(df1['Timezone'])
df1['Nautical_Twilight'] = labelencoder.fit_transform(df1['Nautical_Twilight'])
df1['Wind_Direction'] = labelencoder.fit_transform(df1['Wind_Direction'])
df1['Weather_Condition'] = labelencoder.fit_transform(df1['Weather_Condition'])
df1['Sunrise_Sunset'] = labelencoder.fit_transform(df1['Sunrise_Sunset'])
df1['Weekday'] = labelencoder.fit_transform(df1['Weekday'])
df1['Month'] = labelencoder.fit_transform(df1['Month'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [79]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1886924 entries, 2 to 2246263
Data columns (total 43 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Source                 int64  
 1   TMC                    float64
 2   Severity               float64
 3   Start_Lng              float64
 4   Description            int64  
 5   Wind_Speed(mph)        float64
 6   Astronomical_Twilight  int64  
 7   Start_Lat              float64
 8   Start_Lng              float64
 9   Distance(mi)           float64
 10  Side                   int64  
 11  City                   int64  
 12  County                 int64  
 13  Civil_Twilight         int64  
 14  State                  int64  
 15  Timezone               int64  
 16  Temperature(F)         float64
 17  Humidity(%)            float64
 18  Nautical_Twilight      int64  
 19  Pressure(in)           float64
 20  Visibility(mi)         float64
 21  Wind_Direction         int64  
 22  Weather_Condition 

In [80]:
# checking the severity values

df['Severity'].value_counts()

2.0    1993210
3.0     887547
4.0      92006
1.0        968
Name: Severity, dtype: int64

In [81]:
# Allocating the columns 

y = df1['Severity']
X = df1.drop(['Severity'],axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
X = df1

In [83]:
df

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Year,Month,Day,Hour,Weekday,Time_Duration(min)
0,A-1,MapQuest,201.0,3.0,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.010,Right lane blocked due to accident on I-70 Eas...,NaN,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Night,Night,Night,Night,2016.0,Feb,8.0,5.0,Mon,314.0
1,A-2,MapQuest,201.0,2.0,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.010,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Night,Night,Night,Day,2016.0,Feb,8.0,6.0,Mon,30.0
2,A-3,MapQuest,201.0,2.0,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.010,Accident on OH-32 State Route 32 Westbound at ...,NaN,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Night,Night,Day,Day,2016.0,Feb,8.0,6.0,Mon,30.0
3,A-4,MapQuest,201.0,3.0,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.010,Accident on I-75 Southbound at Exits 52 52B US...,NaN,I-75 S,R,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Night,Day,Day,Day,2016.0,Feb,8.0,7.0,Mon,30.0
4,A-5,MapQuest,201.0,2.0,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.010,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,Miamisburg Centerville Rd,R,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Day,Day,Day,Day,2016.0,Feb,8.0,7.0,Mon,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2974330,A-2974354,Bing,NaN,2.0,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379360,33.99888,-117.37094,0.543,At Market St - Accident.,NaN,Pomona Fwy E,R,Riverside,Riverside,CA,92501,US,US/Pacific,KRAL,2019-08-23 17:53:00,86.0,86.0,40.0,28.92,10.0,W,13.0,0.00,Fair,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Day,Day,Day,Day,2019.0,Aug,23.0,18.0,Fri,29.0
2974331,A-2974355,Bing,NaN,2.0,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148060,32.76555,-117.15363,0.338,At Camino Del Rio/Mission Center Rd - Accident.,NaN,I-8 W,R,San Diego,San Diego,CA,92108,US,US/Pacific,KMYF,2019-08-23 18:53:00,70.0,70.0,73.0,29.39,10.0,SW,6.0,0.00,Fair,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Day,Day,Day,Day,2019.0,Aug,23.0,19.0,Fri,27.0
2974332,A-2974356,Bing,NaN,2.0,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847790,33.77740,-117.85727,0.561,At Glassell St/Grand Ave - Accident. in the ri...,NaN,Garden Grove Fwy,R,Orange,Orange,CA,92866,US,US/Pacific,KSNA,2019-08-23 18:53:00,73.0,73.0,64.0,29.74,10.0,SSW,10.0,0.00,Partly Cloudy,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Day,Day,Day,Day,2019.0,Aug,23.0,19.0,Fri,28.0
2974333,A-2974357,Bing,NaN,2.0,2019-08-23 19:00:21,2019-08-23 19:29:42,33.992460,-118.403020,33.

In [0]:
# Impleminting Encoder 

from keras.utils import np_utils
Y1 = np_utils.to_categorical(y)

In [0]:

Y1 = Y1[:,1:5]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y1, test_size=0.33, random_state=45)

In [100]:
# first neural network with keras tutorial

from numpy import loadtxt

from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

# load the dataset
#dataset = loadtxt('pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (y) variables

# defining the keras model

model = Sequential()
model.add(Dense(12, input_dim=42, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(6, activation='relu'))
model.add(Dense(4, activation='softmax'))
# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=20, batch_size=100)
# evaluate the keras model
_, accuracy = model.evaluate(X,Y1)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/20
1264239/1264239 [==============================] - 31s 25us/step - loss: 0.6449 - accuracy: 0.6775
Epoch 2/20
1264239/1264239 [==============================] - 30s 24us/step - loss: 0.6232 - accuracy: 0.6838
Epoch 3/20
1264239/1264239 [==============================] - 30s 24us/step - loss: 0.6214 - accuracy: 0.6852
Epoch 4/20
1264239/1264239 [==============================] - 33s 26us/step - loss: 0.6210 - accuracy: 0.6856
Epoch 5/20
1264239/1264239 [==============================] - 32s 25us/step - loss: 0.6207 - accuracy: 0.6856
Epoch 6/20
1264239/1264239 [==============================] - 32s 25us/step - loss: 0.6351 - accuracy: 0.6763
Epoch 7/20
1264239/1264239 [==============================] - 32s 25us/step - loss: 0.6561 - accuracy: 0.6633
Epoch 8/20
1264239/1264239 [==============================] - 31s 25us/step - loss: 0.6558 - accuracy: 0.6635
Epoch 9/20
1264239/1264239 [==============================] - 32s 25us/step - loss: 0.6560 - accuracy: 0.6633
Epoch 10/2